In [7]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: entrypoints in c:\users\akshay\anaconda3\lib\site-packages (from altair>=4.2.0->gradio) (0.3)



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!python retrain.py  --bottleneck_dir=bottlenecks --model_dir=model_dir --how_many_training_steps 500  --output_graph=retrained_graph.pb --output_labels=retrained_labels.txt --summaries_dir=retrain_logs --image_dir=image_labels_dir

Looking for images in 'img'
100 bottleneck files created.
2023-04-18 23:16:41.279103: Step 0: Train accuracy = 75.2%
2023-04-18 23:16:41.279103: Step 0: Cross entropy = 0.671839
2023-04-18 23:16:43.319417: Step 0: Validation accuracy = 75.7%
2023-04-18 23:16:44.304638: Step 10: Train accuracy = 75.2%
2023-04-18 23:16:44.304638: Step 10: Cross entropy = 0.571615
2023-04-18 23:16:44.479902: Step 10: Validation accuracy = 74.9%
2023-04-18 23:16:45.642185: Step 20: Train accuracy = 75.2%
2023-04-18 23:16:45.642185: Step 20: Cross entropy = 0.546629
2023-04-18 23:16:45.724297: Step 20: Validation accuracy = 74.5%
2023-04-18 23:16:46.617029: Step 30: Train accuracy = 74.7%
2023-04-18 23:16:46.617029: Step 30: Cross entropy = 0.540641
2023-04-18 23:16:46.708987: Step 30: Validation accuracy = 75.5%
2023-04-18 23:16:47.993743: Step 40: Train accuracy = 75.8%

Instructions for updating:
non-resource variables are not supported in the long term
2023-04-18 23:16:35.619805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use tf.gfile.GFile.
W0418 23:16:35.620878 17040 deprecation.py:350] From retrain.py:247: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
2023-04-18 23:16:35.818023: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
INFO:tensorflow:Summary name cross entropy is illegal; using cross_entropy instead.
I0418 2


2023-04-18 23:16:47.993743: Step 40: Cross entropy = 0.520325
2023-04-18 23:16:48.181004: Step 40: Validation accuracy = 75.5%
2023-04-18 23:16:49.311973: Step 50: Train accuracy = 76.5%
2023-04-18 23:16:49.311973: Step 50: Cross entropy = 0.511854
2023-04-18 23:16:49.422628: Step 50: Validation accuracy = 75.6%
2023-04-18 23:16:50.582448: Step 60: Train accuracy = 75.0%
2023-04-18 23:16:50.582448: Step 60: Cross entropy = 0.526278
2023-04-18 23:16:50.692200: Step 60: Validation accuracy = 74.7%
2023-04-18 23:16:51.563703: Step 70: Train accuracy = 75.2%
2023-04-18 23:16:51.563703: Step 70: Cross entropy = 0.518866
2023-04-18 23:16:51.658644: Step 70: Validation accuracy = 74.3%
2023-04-18 23:16:52.464715: Step 80: Train accuracy = 75.7%
2023-04-18 23:16:52.464715: Step 80: Cross entropy = 0.497453
2023-04-18 23:16:52.542931: Step 80: Validation accuracy = 75.3%
2023-04-18 23:16:53.463369: Step 90: Train accuracy = 76.1%
2023-04-18 23:16:53.463369: Step 90: Cross entropy = 0.502991
20

In [19]:
pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.Successfully installed tabulate-0.9.0



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import sys 
import gradio as gr
def image_classifier(inp):
    pred=["EGFR","KRAS","TP53","LRP1B"]
    prog=["STK11","FAT1","SETBP1","FAT4","KEAP1"]
    st=""
    image_data = tf.gfile.FastGFile(inp, 'rb').read()
    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line in tf.gfile.GFile("labels.txt")]

        # Unpersists graph from file
    with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
        predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})
    
            # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
            if score >=0.30:
                if human_string in pred:
                    st+=human_string+"      "+str(score)+"        "+"Predictive Biomarker"+"\n"
                else:
                    st+=human_string+"      "+str(score)+"        "+"Prognostic Biomarker"+"\n"
        return st
    
demo = gr.Interface(fn=image_classifier, inputs=gr.Textbox(), outputs="text")
demo.launch()



Instructions for updating:
non-resource variables are not supported in the long term
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Instructions for updating:
Use tf.gfile.GFile.
